# **Facial Attendance System**

# Libraries

In [26]:
import cv2
import pandas as pd
import os

# Directory

In [27]:
# Directory to save student images (if necessary)
image_dir = "student_images"
os.makedirs(image_dir, exist_ok=True)

# CSV file to store student details
csv_file = "students.csv"

# Checking No. Of Absentees Of Students

By reading students.csv file and getting no. of students and then comparing with the picture of classroom

In [28]:
# Function to read the registered students from the CSV file
def get_registered_students():
    if not os.path.exists(csv_file):
        print("No registered students found.")
        return None
    df = pd.read_csv(csv_file)
    return df


# Function to detect faces in the classroom image using Haar cascades
def detect_faces_in_classroom(image_path):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return list(faces)



# Function to capture the classroom image
def capture_classroom_image():
    cap = cv2.VideoCapture(0)
    image_path = os.path.join(image_dir, "classroom.jpg")
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return None

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break

        cv2.imshow("Capture Classroom Image - Press 'c' to capture", frame)

        key = cv2.waitKey(1)
        if key == ord('c'):  # Capture image when 'c' key is pressed
            cv2.imwrite(image_path, frame)
            print(f"Classroom image saved at {image_path}")
            break
        elif key == 27:  # Close window when 'Esc' key is pressed
            print("Capture cancelled.")
            break

    cap.release()
    cv2.destroyAllWindows()
    return image_path

def check_attendance():
    # Read the registered students from the CSV file
    df = get_registered_students()
    if df is None:
        return
    
    # Prompt user to capture a new classroom image or provide an existing image path
    choice = input("Would you like to (1) Capture a new classroom image or (2) Provide an existing classroom image path? Enter 1 or 2: ")

    if choice == '1':
        classroom_image_path = capture_classroom_image()
        if classroom_image_path is None:
            print("Classroom image capture failed. Attendance check aborted.")
            return
    elif choice == '2':
        classroom_image_path = input("Enter the full path of the existing classroom image: ")
        if not os.path.isfile(classroom_image_path):
            print("Invalid image path. Please try again.")
            return
    else:
        print("Invalid choice. Please try again.")
        return

    # Detect faces in the classroom image
    detected_faces_count = len(detect_faces_in_classroom(classroom_image_path))
    print(f"Number of faces detected in classroom: {detected_faces_count}")

    # Get the number of registered students
    registered_students_count = len(df)
    print(f"Number of registered students: {registered_students_count}")

    # Calculate the number of absentees
    absentees_count = registered_students_count - detected_faces_count
    absentees_count = max(0, absentees_count)  # Ensure the count is not negative

    print(f"Number of absentees: {absentees_count}")

check_attendance()


Classroom image saved at student_images\classroom.jpg
Number of faces detected in classroom: 1
Number of registered students: 2
Number of absentees: 1


# Mark Attendance in CSV

In [30]:
def mark_attendance():
    df = get_registered_students()
    if df is None:
        return
    
    choice = input("Would you like to (1) Capture a new classroom image or (2) Provide an existing classroom image path? Enter 1 or 2: ")

    if choice == '1':
        classroom_image_path = capture_classroom_image()
        if classroom_image_path is None:
            print("Classroom image capture failed. Attendance marking aborted.")
            return
    elif choice == '2':
        classroom_image_path = input("Enter the full path of the existing classroom image: ")
        if not os.path.isfile(classroom_image_path):
            print("Invalid image path. Please try again.")
            return
    else:
        print("Invalid choice. Please try again.")
        return

    faces = detect_faces_in_classroom(classroom_image_path)

    # Add a new column for attendance
    attendance_col = f"Day_{df.shape[1] - 2}"  # Subtract 2 for Roll Number and Name columns
    df[attendance_col] = "Absent"
    
    for face in faces:
        # Compute centroid of the face rectangle
        x, y, w, h = face
        cx = x + w // 2
        cy = y + h // 2

        # Check if the centroid is within any registered student's face rectangle
        for index, row in df.iterrows():
            roll_number = row['Roll Number']
            name = row['Name']
            image_path = row['Image Path']

            registered_faces = detect_faces_in_classroom(image_path)
            for rx, ry, rw, rh in registered_faces:
                rcx = rx + rw // 2
                rcy = ry + rh // 2
                if abs(rcx - cx) < rw // 2 and abs(rcy - cy) < rh // 2:
                    df.at[index, attendance_col] = "Present"
                    break  # Break inner loop as student is found
            else:
                continue  # Continue to the next registered student if the current face is not matched
    
    # Save the updated attendance
    df.to_csv(csv_file, index=False)
    print("Attendance marked successfully.")

mark_attendance()


Classroom image saved at student_images\classroom.jpg
Attendance marked successfully.
